<a href="https://colab.research.google.com/github/masifkhan3/Genetic-II/blob/main/Genetic_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install transformers faiss-cpu datasets pandas


In [3]:
import pandas as pd

# Load your dataset
data = pd.read_csv("/content/genetic-Final.csv")

# Combine columns to form a single "text" field for retrieval purposes
data["text"] = (
    "Disease Name: " + data["Disease Name"] +
    "\nGene(s) Involved: " + data["Gene(s) Involved"] +
    "\nInheritance Pattern: " + data["Inheritance Pattern"] +
    "\nSymptoms: " + data["Symptoms"] +
    "\nSeverity Level: " + data["Severity Level"] +
    "\nRisk Assessment: " + data["Risk Assessment"] +
    "\nTreatment Options: " + data["Treatment Options"] +
    "\nSuggested Medical Tests: " + data["Suggested Medical Tests"] +
    "\nMinimum Values for Medical Tests: " + data["Minimum Values for Medical Tests"] +
    "\nEmergency Treatment: " + data["Emergency Treatment"]
)
texts = data["text"].tolist()


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Function to generate embeddings
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# Generate embeddings for all texts
embeddings = [embed_text(text) for text in texts]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [5]:
# Fill NaN values with an empty string and ensure all values are strings
data["text"] = data["text"].fillna("").astype(str)
texts = data["text"].tolist()


In [6]:
# Generate embeddings for all texts
embeddings = [embed_text(text) for text in texts]


In [8]:
import pandas as pd

# Load your dataset
data = pd.read_csv("/content/genetic-Final.csv")

# Combine columns to form a single "text" field for retrieval purposes
data["text"] = (
    "Disease Name: " + data["Disease Name"].fillna("") +
    "\nGene(s) Involved: " + data["Gene(s) Involved"].fillna("") +
    "\nInheritance Pattern: " + data["Inheritance Pattern"].fillna("") +
    "\nSymptoms: " + data["Symptoms"].fillna("") +
    "\nSeverity Level: " + data["Severity Level"].fillna("") +
    "\nRisk Assessment: " + data["Risk Assessment"].fillna("") +
    "\nTreatment Options: " + data["Treatment Options"].fillna("") +
    "\nSuggested Medical Tests: " + data["Suggested Medical Tests"].fillna("") +
    "\nMinimum Values for Medical Tests: " + data["Minimum Values for Medical Tests"].fillna("") +
    "\nEmergency Treatment: " + data["Emergency Treatment"].fillna("")
)

# Ensure all entries in `texts` are strings
texts = data["text"].fillna("").astype(str).tolist()

from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Function to generate embeddings
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# Generate embeddings for all texts
embeddings = [embed_text(text) for text in texts]

# Initialize FAISS index
embedding_dim = embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings))


In [10]:
# Function to get user input and generate response
def ask_user():
    # Ask for input on diseases, symptoms, or medical tests
    print("Welcome to the Medical Query Assistant!")
    disease = input("Enter the disease name (or press Enter to skip): ")
    symptoms = input("Enter symptoms (or press Enter to skip): ")
    medical_test = input("Enter medical test details (or press Enter to skip): ")

    # Combine the inputs into a query
    query = ""
    if disease:
        query += f"Disease Name: {disease}\n"
    if symptoms:
        query += f"Symptoms: {symptoms}\n"
    if medical_test:
        query += f"Medical Test: {medical_test}\n"

    # Check if any query info was provided
    if not query.strip():
        print("No input provided. Please enter at least one detail.")
        return ask_user()  # Recursively call the function to get input

    # Generate a response based on the query
    response = retrieve_and_generate(query)
    print("Generated Response:", response)

# Function to retrieve and generate response using the query
def retrieve_and_generate(query, top_k=3):
    # Generate query embedding
    query_embedding = embed_text(query).reshape(1, -1)

    # Retrieve top-k similar texts
    distances, indices = index.search(query_embedding, top_k)
    retrieved_texts = [texts[i] for i in indices[0]]

    # Prepare input for generation by combining retrieved texts
    context = "\n\n".join(retrieved_texts) + "\n\nQuestion: " + query

    # Generate answer
    inputs = generator_tokenizer(context, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator_model.generate(**inputs, max_length=150, num_return_sequences=1)

    # Decode the output
    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Start the interactive query session
ask_user()


Welcome to the Medical Query Assistant!
Enter the disease name (or press Enter to skip): gsd
Enter symptoms (or press Enter to skip): 
Enter medical test details (or press Enter to skip): 


NameError: name 'generator_tokenizer' is not defined

In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load a generator model and tokenizer
generator_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
generator_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")

# Function to retrieve similar entries and generate a response
def retrieve_and_generate(query, top_k=3):
    # Generate query embedding
    query_embedding = embed_text(query).reshape(1, -1)

    # Retrieve top-k similar texts
    distances, indices = index.search(query_embedding, top_k)
    retrieved_texts = [texts[i] for i in indices[0]]

    # Prepare input for generation by combining retrieved texts
    context = "\n\n".join(retrieved_texts) + "\n\nQuestion: " + query

    # Generate answer
    inputs = generator_tokenizer(context, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator_model.generate(**inputs, max_length=150, num_return_sequences=1)

    # Decode the output
    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Function to get user input and generate response
def ask_user():
    # Ask for input on diseases, symptoms, or medical tests
    print("Welcome to the Medical Query Assistant!")
    disease = input("Enter the disease name (or press Enter to skip): ")
    symptoms = input("Enter symptoms (or press Enter to skip): ")
    medical_test = input("Enter medical test details (or press Enter to skip): ")

    # Combine the inputs into a query
    query = ""
    if disease:
        query += f"Disease Name: {disease}\n"
    if symptoms:
        query += f"Symptoms: {symptoms}\n"
    if medical_test:
        query += f"Medical Test: {medical_test}\n"

    # Check if any query info was provided
    if not query.strip():
        print("No input provided. Please enter at least one detail.")
        return ask_user()  # Recursively call the function to get input

    # Generate a response based on the query
    response = retrieve_and_generate(query)
    print("Generated Response:", response)

# Start the interactive query session
ask_user()


Welcome to the Medical Query Assistant!
Enter the disease name (or press Enter to skip): Trisomy 18
Enter symptoms (or press Enter to skip): 
Enter medical test details (or press Enter to skip): 
Generated Response: Disease Name: Trisomy 18                Gene(s) Involved: No specific gene; chromosomal anomalySourceFileInheritance Pattern: SourceFileSymptoms: Severe developmental delays, heart defects, clenched hands�Severity Level: Moderate to Severe                Risk Assessment: High                Treatment Options: Supportive therapies (speech, occupational, physical therapy), Regular health monitoring,Educational interventions                Suggested Medical Tests:   Karyotype analysis,Palliative care options for severe cases�━━━──────────────────────────────────────────────────────────────┻━────────┼━━────┾━━┻─────────────┼────────────


In [19]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load a generator model and tokenizer
generator_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
generator_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")

# Function to retrieve similar entries and generate a response
def retrieve_and_generate(query, top_k=3):
    # Generate query embedding
    query_embedding = embed_text(query).reshape(1, -1)

    # Retrieve top-k similar texts
    distances, indices = index.search(query_embedding, top_k)
    retrieved_texts = [texts[i] for i in indices[0]]

    # Prepare input for generation by combining retrieved texts
    context = "\n\n".join(retrieved_texts) + "\n\nQuestion: " + query

    # Generate answer
    inputs = generator_tokenizer(context, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator_model.generate(**inputs, max_length=150, num_return_sequences=1)

    # Decode the output
    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Function to get user input and generate response
def ask_user():
    # Ask for patient information and medical details
    print("Welcome to the Medical Query Assistant!")
    patient_name = input("Please enter the patient's name: ")
    patient_sex = input("Please enter the patient's sex (e.g., Male, Female, Other): ")
    patient_age = input("Please enter the patient's age (in years): ")

    disease = input("\nEnter the disease name if known (or press Enter to skip): ")
    symptoms = input("Describe any symptoms (separate multiple symptoms with commas, or press Enter to skip): ")
    medical_test = input("Enter details about any relevant medical tests conducted (or press Enter to skip): ")

    # Combine the inputs into a query with patient information
    query = f"Patient Name: {patient_name}\nSex: {patient_sex}\nAge: {patient_age} years\n"
    if disease:
        query += f"Disease Name: {disease}\n"
    if symptoms:
        query += f"Symptoms: {symptoms}\n"
    if medical_test:
        query += f"Medical Test: {medical_test}\n"

    # Check if any query info was provided
    if not disease and not symptoms and not medical_test:
        print("No specific disease, symptoms, or medical test information provided. Please enter at least one detail.")
        return ask_user()  # Recursively call the function to get input

    # Generate a response based on the query
    response = retrieve_and_generate(query)
    print("\nGenerated Response:", response)

# Start the interactive query session
ask_user()


Welcome to the Medical Query Assistant!
Please enter the patient's name: asif
Please enter the patient's sex (e.g., Male, Female, Other): male
Please enter the patient's age (in years): 40

Enter the disease name if known (or press Enter to skip): Trisomy 18
Describe any symptoms (separate multiple symptoms with commas, or press Enter to skip): 
Enter details about any relevant medical tests conducted (or press Enter to skip): 

Generated Response: Disease Name: Trisomy 18                Gene(s) Involved: No specific gene; chromosomal anomalySourceFileInheritance Pattern: �Symptoms: Severe developmental delays, heart defects, clenched hands�Severity Level: Moderate to Severe                Risk Assessment: High                Treatment Options: Supportive therapies (speech, occupational, physical therapy), Regular health monitoring,Educational interventions                Suggested Medical Tests:   Karyotype analysis                Minimum Values for Medical Tests (Karyotype) through k